In [13]:
import os
import glob
import cv2
import tensorflow as tf

try:
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except (AssertionError, ValueError, RuntimeError):
    raise RuntimeError('Unable to disable GPU')

    
import matplotlib.pyplot as plt
import glob
import numpy as np
import random

from PIL import Image

In [14]:
dim = 64

def read_img(file):
    img = tf.io.read_file(file)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [dim, dim])
    img = img/127.5-1
    return img

In [15]:
def conv_model(num_blocks, rate, dim):
    
    input = tf.keras.layers.Input(shape=(dim,dim,3))
    
    x = tf.keras.layers.Conv2D(16, (16,16), strides=1)(input)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
        
    for i in range(num_blocks-1):
        x = tf.keras.layers.Conv2D(8, (8,8), strides=1)(x)
        x = tf.keras.layers.Dropout(rate)(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('relu')(x)
        
    x = tf.keras.layers.Conv2D(filters=2, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)
    
    model = tf.keras.Model(inputs=input, outputs=predictions)
    print(model.summary())
    
    return model


In [16]:
num_blocks = 3
dropout = .1
model = conv_model(num_blocks,.1, dim)
model.load_weights('mask_classification_model_3_50.h5')
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 49, 49, 16)        12304     
_________________________________________________________________
dropout_16 (Dropout)         (None, 49, 49, 16)        0         
_________________________________________________________________
batch_normalization_16 (Batc (None, 49, 49, 16)        64        
_________________________________________________________________
activation_16 (Activation)   (None, 49, 49, 16)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 42, 42, 8)         8200      
_________________________________________________________________
dropout_17 (Dropout)         (None, 42, 42, 8)         0   

In [17]:
###0 is mask, 1 no mask
biden = read_img('biden.png')
trump = read_img('trump.png')

biden_pred = model.predict(np.expand_dims(biden,axis=0))
biden_pred = np.argmax(biden_pred,axis=1)

trump_pred = model.predict(np.expand_dims(trump,axis=0))
trump_pred = np.argmax(trump_pred,axis=1)

print(f"Biden: {biden_pred} and Trump: {trump_pred}")

Biden: [0] and Trump: [1]
